In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv('Data_for_UCI_named.csv')

In [5]:
df = data.copy()

In [6]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [7]:
df = df.drop('stab',axis=1)
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [8]:
X = df.drop('stabf',axis=1)
y = df['stabf']

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalized_x_train = pd.DataFrame(scaler.fit_transform(x_train,y_train),columns = x_train.columns)
normalized_x_test = pd.DataFrame(scaler.fit_transform(x_test),columns = x_test.columns)

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_score,f1_score,recall_score,classification_report,accuracy_score


In [29]:
#Question14
forest = RandomForestClassifier(random_state=1)
forest.fit(normalized_x_train,y_train)
pred1 =forest.predict(normalized_x_test)
print('Accuracy:{}'.format(round(accuracy_score(y_test,pred1),4)))

Accuracy:0.928


In [30]:
#Question15
xgb = XGBClassifier(random_state=1,learning_rate = 0.1, max_depth = 3,eval_metric='error')
xgb.fit(normalized_x_train,y_train)
pred3 =xgb.predict(normalized_x_test)
print('Accuracy:{}'.format(round(accuracy_score(y_test,pred3),4)))

Accuracy:0.919


In [31]:
Igbm = LGBMClassifier(random_state=2)
Igbm.fit(normalized_x_train,y_train)
pred4 =Igbm.predict(normalized_x_test)
print('Accuracy:{}'.format(round(accuracy_score(y_test,pred4),4)))

Accuracy:0.9365


In [22]:
#Question17 and 18: from cell 14 to cell 17
tree = ExtraTreesClassifier(random_state=3)
tree.fit(normalized_x_train,y_train)
pred2 =forest.predict(normalized_x_test)
print('Accuracy:{}'.format(round(accuracy_score(y_test,pred2),4)))

Accuracy:0.928


In [23]:
n_estimators = [50,100,300,500,1000]
min_samples_split = [2,3,5,7,9]
min_samples_leaf = [1,2,4,6,8]
max_features = ['auto','sqrt','log2',None]
hyperparameter_grid = {'n_estimators': n_estimators,
                    'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                    'max_features': max_features}

In [24]:
from sklearn.model_selection import RandomizedSearchCV
rsv = RandomizedSearchCV(tree, hyperparameter_grid,cv=5,n_iter=10,scoring='accuracy',n_jobs=-1,verbose=1,random_state=1)
search = rsv.fit(normalized_x_train,y_train)
search.best_params_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

In [25]:
tree = ExtraTreesClassifier(**search.best_params_, random_state=1)
tree.fit(normalized_x_train,y_train)
pred5 = tree.predict(normalized_x_test)
print('Accirancy: {}'.format(accuracy_score(y_test,pred5)))

Accirancy: 0.9285
